<b>Pruning</b>

In [ ]:
import numpy as np
import torch

seed = 100
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [ ]:
# Dataset

from torchvision import datasets, transforms
import torch.utils as utils

transform = transforms.Compose([
    transforms.ToTensor()])

dataset_train = datasets.FashionMNIST(
    '~/fashion-mnist', 
    train=True, 
    download=True, 
    transform=transform)
dataset_test  = datasets.FashionMNIST(
    '~/fashion-mnist', 
    train=False, 
    download=True, 
    transform=transform)

batch_size = 1000

dataloader_train = utils.data.DataLoader(dataset_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=4)
dataloader_test  = utils.data.DataLoader(dataset_test,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=4)

Extracting /root/fashion_mnist/FashionMNIST/raw/train-images-idx3-ubyte.gz to /root/fashion_mnist/FashionMNIST/raw


Extracting /root/fashion_mnist/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /root/fashion_mnist/FashionMNIST/raw



Extracting /root/fashion_mnist/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /root/fashion_mnist/FashionMNIST/raw


Extracting /root/fashion_mnist/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /root/fashion_mnist/FashionMNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
# Network

import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv = nn.Sequential(OrderedDict([
        ("conv1", nn.Conv2d(1, 4, kernel_size=5)),
        ("relu1", nn.ReLU()),
        ("pool1", nn.MaxPool2d(2)),
        ("conv2", nn.Conv2d(4, 8, kernel_size=5)),
        ("relu2", nn.ReLU()),
        ("pool2", nn.MaxPool2d(2)),
    ]))
    self.fc = nn.Sequential(OrderedDict([
        ("fc1"  , nn.Linear(8 * 4 * 4, 100)),
        ("relu1", nn.ReLU()),
        ("fc2"  , nn.Linear(100, 10)),
    ]))

  def forward(self, x1):
    x2 = self.conv(x1)
    x3 = x2.view(x2.size()[0], -1)
    y  = self.fc(x3)
    return y

In [ ]:
# Training

from torch import optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

for i in range(20):
  print(f"EPOCH: {i+1}")

  ### Train ###
  model.train()
  for x, t in dataloader_train:
    x = x.to(device)
    t = t.to(device)
    model.zero_grad()
    y = model(x)
    loss = criterion(y, t)
    loss.backward()
    optimizer.step()

  model.eval()
  sum_loss = 0.0
  sum_correct = 0
  sum_iter = 0
  for x, t in dataloader_train:
    x = x.to(device)
    t = t.to(device)
    y = model(x)
    loss = criterion(y, t)
    _, predicted = y.max(1)
    sum_loss += loss.cpu().detach().numpy()
    sum_correct += (predicted == t).sum().item()
    sum_iter += 1
  print(f"  train loss: {sum_loss/sum_iter}")
  print(f"  train acc : {sum_correct/(sum_iter*batch_size)}")

  ### Test ###
  model.eval()
  sum_loss = 0.0
  sum_correct = 0
  sum_iter = 0
  for x, t in dataloader_test:
    x = x.to(device)
    t = t.to(device)
    y = model(x)
    loss = criterion(y, t)
    _, predicted = y.max(1)
    sum_loss += loss.cpu().detach().numpy()
    sum_correct += (predicted == t).sum().item()
    sum_iter += 1
  print(f"  test  loss: {sum_loss/sum_iter}")
  print(f"  test  acc : {sum_correct/(sum_iter*batch_size)}")

EPOCH: 1
  train loss: 0.9168381800254186
  train acc : 0.6813833333333333
  test  loss: 0.9292261481285096
  test  acc : 0.6743
EPOCH: 2
  train loss: 0.6959822833538055
  train acc : 0.7428
  test  loss: 0.7132895767688752
  test  acc : 0.7349
EPOCH: 3
  train loss: 0.6218226710955302
  train acc : 0.7685
  test  loss: 0.6417523205280304
  test  acc : 0.7601
EPOCH: 4
  train loss: 0.5811235576868057
  train acc : 0.78465
  test  loss: 0.6022679567337036
  test  acc : 0.7785
EPOCH: 5
  train loss: 0.5509732181827227
  train acc : 0.79445
  test  loss: 0.5717193424701691
  test  acc : 0.7863
EPOCH: 6
  train loss: 0.5283349002401034
  train acc : 0.8029
  test  loss: 0.549051222205162
  test  acc : 0.7966
EPOCH: 7
  train loss: 0.5043047547340394
  train acc : 0.8190333333333333
  test  loss: 0.5297306180000305
  test  acc : 0.8099
EPOCH: 8
  train loss: 0.4892908498644829
  train acc : 0.82265
  test  loss: 0.5142027407884597
  test  acc : 0.8144
EPOCH: 9
  train loss: 0.4715465332070

In [ ]:
# Check Parameter after Training

print(model.conv.conv1.weight)

Parameter containing:
tensor([[[[-3.6468e-01, -4.6546e-02, -3.0542e-01, -2.1761e-01, -1.0700e-01],
          [ 2.5952e-02, -1.4520e-01, -2.7217e-01, -2.3535e-01, -2.6463e-01],
          [-1.3805e-01,  1.8698e-01,  2.0523e-01,  1.4153e-01, -6.8428e-02],
          [ 2.6195e-01,  3.4516e-01,  2.4391e-01, -5.4122e-02, -5.5166e-02],
          [-5.8439e-02,  2.6185e-01,  1.3891e-01,  2.4380e-01,  9.7636e-03]]],


        [[[ 2.3045e-01,  5.9099e-02,  1.1136e-01, -2.8380e-02,  1.3503e-01],
          [ 5.7270e-02, -1.7238e-01,  2.3187e-01,  8.8662e-02, -9.4497e-02],
          [-2.1362e-01, -2.4744e-02,  1.2921e-01, -5.3550e-02,  2.6260e-01],
          [-1.9787e-01, -5.2112e-01,  6.4715e-02,  9.3146e-02,  1.0119e-01],
          [-2.4131e-01, -5.5181e-01, -1.2298e-02,  3.1706e-01,  1.5635e-01]]],


        [[[ 8.4349e-02,  2.6439e-01,  1.2919e-01,  1.2679e-01,  2.2132e-01],
          [ 1.1629e-01,  2.1276e-01,  2.8475e-02,  2.9080e-01,  2.7386e-01],
          [-4.6168e-02,  1.6856e-01,  1.0014e-

In [ ]:
# Pruning

import torch.nn.utils.prune as prune

parameters_to_prune = (
    (model.conv.conv1, "weight"),
    (model.conv.conv2, "weight"),
    (model.fc.fc1,     "weight"),
    (model.fc.fc2,     "weight"),
)

prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.9,
)

In [ ]:
# Check Parameter after Pruning

print(model.conv.conv1.weight)

tensor([[[[-0.3647, -0.0000, -0.3054, -0.2176, -0.0000],
          [ 0.0000, -0.0000, -0.2722, -0.2354, -0.2646],
          [-0.0000,  0.1870,  0.2052,  0.0000, -0.0000],
          [ 0.2619,  0.3452,  0.2439, -0.0000, -0.0000],
          [-0.0000,  0.2619,  0.0000,  0.2438,  0.0000]]],


        [[[ 0.2304,  0.0000,  0.0000, -0.0000,  0.0000],
          [ 0.0000, -0.1724,  0.2319,  0.0000, -0.0000],
          [-0.2136, -0.0000,  0.0000, -0.0000,  0.2626],
          [-0.1979, -0.5211,  0.0000,  0.0000,  0.0000],
          [-0.2413, -0.5518, -0.0000,  0.3171,  0.1564]]],


        [[[ 0.0000,  0.2644,  0.0000,  0.0000,  0.2213],
          [ 0.0000,  0.2128,  0.0000,  0.2908,  0.2739],
          [-0.0000,  0.1686,  0.0000,  0.2816,  0.2578],
          [ 0.2189,  0.2649,  0.2041,  0.0000,  0.0000],
          [ 0.3242,  0.2487, -0.0000,  0.2174,  0.0000]]],


        [[[-0.2853,  0.3724,  0.2013,  0.3021,  0.1529],
          [-0.2579,  0.3922,  0.0000, -0.0000,  0.3151],
          [-0.0000,

In [ ]:
# Retraining

for i in range(20):
  print(f"EPOCH: {i+1}")

  ### Train ###
  model.train()
  for x, t in dataloader_train:
    x = x.to(device)
    t = t.to(device)
    model.zero_grad()
    y = model(x)
    loss = criterion(y, t)
    loss.backward()
    optimizer.step()

  model.eval()
  sum_loss = 0.0
  sum_correct = 0
  sum_iter = 0
  for x, t in dataloader_train:
    x = x.to(device)
    t = t.to(device)
    y = model(x)
    loss = criterion(y, t)
    _, predicted = y.max(1)
    sum_loss += loss.cpu().detach().numpy()
    sum_correct += (predicted == t).sum().item()
    sum_iter += 1
  print(f"  train loss: {sum_loss/sum_iter}")
  print(f"  train acc : {sum_correct/(sum_iter*batch_size)}")

  ### Test ###
  model.eval()
  sum_loss = 0.0
  sum_correct = 0
  sum_iter = 0
  for x, t in dataloader_test:
    x = x.to(device)
    t = t.to(device)
    y = model(x)
    loss = criterion(y, t)
    _, predicted = y.max(1)
    sum_loss += loss.cpu().detach().numpy()
    sum_correct += (predicted == t).sum().item()
    sum_iter += 1
  print(f"  test  loss: {sum_loss/sum_iter}")
  print(f"  test  acc : {sum_correct/(sum_iter*batch_size)}")

EPOCH: 1
  train loss: 0.6591822574536006
  train acc : 0.7839166666666667
  test  loss: 0.6710086345672608
  test  acc : 0.7812
EPOCH: 2
  train loss: 0.5431726887822151
  train acc : 0.8059333333333333
  test  loss: 0.5602730333805084
  test  acc : 0.8051
EPOCH: 3
  train loss: 0.49998450875282285
  train acc : 0.8208333333333333
  test  loss: 0.5202153772115707
  test  acc : 0.8172
EPOCH: 4
  train loss: 0.47718850324551265
  train acc : 0.8287833333333333
  test  loss: 0.4997991681098938
  test  acc : 0.8246
EPOCH: 5
  train loss: 0.46140354375044507
  train acc : 0.8349166666666666
  test  loss: 0.4848212540149689
  test  acc : 0.829
EPOCH: 6
  train loss: 0.45123519698778786
  train acc : 0.8372666666666667
  test  loss: 0.4758285492658615
  test  acc : 0.8297
EPOCH: 7
  train loss: 0.44273183196783067
  train acc : 0.8429333333333333
  test  loss: 0.4687992066144943
  test  acc : 0.8334
EPOCH: 8
  train loss: 0.4351958925525347
  train acc : 0.8467833333333333
  test  loss: 0.46

In [ ]:
# Check Parameter after Retraining

print(model.conv.conv1.weight)

tensor([[[[-0.5079, -0.0000, -0.3723, -0.3363, -0.0000],
          [ 0.0000, -0.0000, -0.3005, -0.3034, -0.2527],
          [-0.0000,  0.2120,  0.3004,  0.0000, -0.0000],
          [ 0.3675,  0.3830,  0.3463, -0.0000, -0.0000],
          [-0.0000,  0.3710,  0.0000,  0.3527,  0.0000]]],


        [[[ 0.2915,  0.0000,  0.0000, -0.0000,  0.0000],
          [ 0.0000, -0.1831,  0.3629,  0.0000, -0.0000],
          [-0.2612, -0.0000,  0.0000, -0.0000,  0.4544],
          [-0.2065, -0.5657,  0.0000,  0.0000,  0.0000],
          [-0.3400, -0.7235, -0.0000,  0.5565,  0.3619]]],


        [[[ 0.0000,  0.4656,  0.0000,  0.0000,  0.4587],
          [ 0.0000,  0.3308,  0.0000,  0.5987,  0.5134],
          [-0.0000,  0.3225,  0.0000,  0.5650,  0.4393],
          [ 0.3406,  0.4398,  0.5003,  0.0000, -0.0000],
          [ 0.4481,  0.3855, -0.0000,  0.4062,  0.0000]]],


        [[[-0.1732,  0.4676,  0.4230,  0.5622,  0.2874],
          [-0.2715,  0.4663,  0.0000, -0.0000,  0.4697],
          [-0.0000,

参考<br>
https://pytorch.org/tutorials/intermediate/pruning_tutorial.html<br>
https://pytorch.org/docs/master/generated/torch.nn.utils.prune.global_unstructured.html